In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI()

In [7]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing tools and frameworks for automated testing of software applications. It can assist in writing test scripts, running tests, and analyzing test results to ensure the quality and stability of the software. Langsmith can also help identify and prioritize test cases based on the specific requirements and functionalities of the application, making the testing process more efficient and effective. Additionally, Langsmith can support continuous integration and deployment practices, helping teams to automate testing as part of their development workflow.', response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 15, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are world class technical documentation writer.'),
    ("user", "{input}"),
])

In [12]:
chain = prompt | llm

In [14]:
chain.invoke({'input': 'how can langsmith help with testing?'})

AIMessage(content='Langsmith can help with testing in various ways:\n\n1. Automation: Langsmith can be used to automate testing tasks, such as generating test data, running test scripts, and comparing actual results with expected results. This can help in accelerating the testing process and ensuring consistency in test execution.\n\n2. Test Data Generation: Langsmith can be used to generate synthetic test data for different scenarios and conditions. This can help in testing the application with a variety of input data to uncover potential issues and edge cases.\n\n3. Performance Testing: Langsmith can be used to simulate different types of user interactions and test the performance of the application under varying load conditions. This can help in identifying performance bottlenecks and optimizing the application for better scalability.\n\n4. API Testing: Langsmith can be used to automate API testing by sending requests to APIs, validating responses, and checking for expected behavior

In [5]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [6]:
chain = prompt | llm | output_parser

In [17]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can assist with testing in several ways:\n\n1. Test Automation: Langsmith can help automate the testing process by generating test scripts for various scenarios. This can save time and reduce the potential for human error in manual testing.\n\n2. Test Data Generation: Langsmith can create realistic and diverse test data sets to ensure thorough testing coverage. This can help identify potential edge cases and uncover hidden bugs.\n\n3. Performance Testing: Langsmith can simulate large numbers of virtual users to test the performance and scalability of a system. This can help identify bottlenecks and optimize system performance.\n\n4. Security Testing: Langsmith can assist in identifying vulnerabilities and potential security risks in an application. It can simulate various attack scenarios to help improve the overall security posture of the system.\n\n5. Regression Testing: Langsmith can help automate the regression testing process by re-running tests on new builds or version

In [7]:
from langchain_community.document_loaders import WebBaseLoader
import os

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [15]:
os.getenv('ES_PASSWORD')

'Ecstel8488*'

In [21]:
from langchain_elasticsearch import ElasticsearchStore
import os

index_name = 'quickstart_index'

elastic_vector_search = ElasticsearchStore(
    embedding=embeddings,
    index_name=index_name,
    es_url=os.getenv('ES_URL'),
)

In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter()

documents = text_spliter.split_documents(docs)

In [40]:
vector = ElasticsearchStore.from_documents(
    documents,
    embeddings,
    es_url=os.getenv("ES_URL"),
    index_name=index_name,
)

In [42]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    "Answer the following question based only on the provided context:\n"
    "<context>\n"
    "{context}\n"
    "<context>\n"
    "Question: {input}"
)

# 문서 리스트를 모델에게 전달하는 체인입니다.
document_chain = create_stuff_documents_chain(llm, prompt)

In [43]:
from langchain.chains import create_retrieval_chain


# 벡터 저장소를 검색하는 부품을 반환합니다.
retriever = vector.as_retriever()

# 검색기를 체인에 추가합니다.
retriever_chain = create_retrieval_chain(retriever, document_chain)

In [45]:
response = retriever_chain.invoke({"input": "how can langsmith help with testing"})

print(response["answer"])

In [47]:
response

{'input': 'how can langsmith help with testing',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly importan

In [50]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation"),
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [52]:
from langchain_core.messages import HumanMessage, AIMessage

# AI가 예라고 대답하였고, 사용자의 후속 질문에 답변하는 상황을 가정합니다.
chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!"),
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM appli

In [53]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

# 검색 체인은 문서 리스트를 반환합니다.
# 이 문서 리스트를 문서를 참고하여 답변하는 체인과 연결합니다.
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [55]:
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how",
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly u